In [ ]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from collections import Counter
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split, KFold, StratifiedKFold
from lightgbm import LGBMClassifier
from sklearn.metrics import roc_auc_score
import numpy as np

In [200]:
df_train = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')

In [ ]:
df_train.head()

,ID,Gender,Age,Region_Code,Occupation,Channel_Code,Vintage,Credit_Product,Avg_Account_Balance,Is_Active,Is_Lead
0,NNVBBKZB,Female,73,RG268,Other,X3,43,No,1045696,No,0
1,IDD62UNG,Female,30,RG277,Salaried,X1,32,No,581988,No,0
2,HD3DSEMC,Female,56,RG268,Self_Employed,X3,26,No,1484315,Yes,0
3,BF3NC7KV,Male,34,RG270,Salaried,X1,19,No,470454,No,0
4,TEASRWXV,Female,30,RG282,Salaried,X1,33,No,886787,No,0


In [201]:
# co-occurence based features
# region code vs channel

matrix1 = dict()
region = df_train['Region_Code'].values
for i in region:
  matrix1[i] = [0,0,0,0]
region_test = df_test['Region_Code'].values
for i in region_test:
  matrix1[i] = [0,0,0,0]

channel = df_train['Channel_Code'].values

for i,j in zip(channel,region):
  if i == 'X1':
    matrix1[j][0] += 1
  elif i == 'X2':
    matrix1[j][1] += 1
  elif i == 'X3':
    matrix1[j][2] += 1
  elif i == 'X4':
    matrix1[j][3] += 1

channel_test = df_test['Channel_Code'].values

for i,j in zip(channel_test,region_test):
  if i == 'X1':
    matrix1[j][0] += 1
  elif i == 'X2':
    matrix1[j][1] += 1
  elif i == 'X3':
    matrix1[j][2] += 1
  elif i == 'X4':
    matrix1[j][3] += 1


#normalizing the features row-wise

new_feat_train = []
new_feat_test = []

for i in region:
  new_feat_train.append(matrix1[i])

for i in region_test:
  new_feat_test.append(matrix1[i])

for i in range(len(new_feat_train)):
  total = 0
  for j in range(len(new_feat_train[i])):
    total += new_feat_train[i][j]
  
  for j in range(len(new_feat_train[i])):
    new_feat_train[i][j] /= total

for i in range(len(new_feat_test)):
  total = 0
  for j in range(len(new_feat_test[i])):
    total += new_feat_test[i][j]
  
  for j in range(len(new_feat_test[i])):
    new_feat_test[i][j] /= total


new_feat_train = np.array(new_feat_train)
new_feat_test = np.array(new_feat_test)

#drop last column to prevent correlation
new_feat_train = new_feat_train[:, :-1]
new_feat_test = new_feat_test[:, :-1]

In [203]:
# co-occurence based features
# Occupation vs Channel_Code

matrix1 = dict()
occupation = df_train['Occupation'].values
for i in occupation:
  matrix1[i] = [0,0,0,0]
occupation_test = df_test['Occupation'].values
for i in occupation_test:
  matrix1[i] = [0,0,0,0]

channel = df_train['Channel_Code'].values

for i,j in zip(channel,occupation):
  if i == 'X1':
    matrix1[j][0] += 1
  elif i == 'X2':
    matrix1[j][1] += 1
  elif i == 'X3':
    matrix1[j][2] += 1
  elif i == 'X4':
    matrix1[j][3] += 1


channel_test = df_test['Channel_Code'].values

for i,j in zip(channel_test,occupation_test):
  if i == 'X1':
    matrix1[j][0] += 1
  elif i == 'X2':
    matrix1[j][1] += 1
  elif i == 'X3':
    matrix1[j][2] += 1
  elif i == 'X4':
    matrix1[j][3] += 1


#normalizing the features row-wise
new_feat_train2 = []
new_feat_test2 = []

for i in occupation:
  new_feat_train2.append(matrix1[i])

for i in occupation_test:
  new_feat_test2.append(matrix1[i])


for i in range(len(new_feat_train2)):
  total = 0
  for j in range(len(new_feat_train2[i])):
    total += new_feat_train2[i][j]
  
  for j in range(len(new_feat_train2[i])):
    new_feat_train2[i][j] /= total

for i in range(len(new_feat_test2)):
  total = 0
  for j in range(len(new_feat_test2[i])):
    total += new_feat_test2[i][j]
  
  for j in range(len(new_feat_test2[i])):
    new_feat_test2[i][j] /= total

new_feat_train2 = np.array(new_feat_train2)
new_feat_test2 = np.array(new_feat_test2)

#drop last column to prevent correlation
new_feat_train2 = new_feat_train2[:, :-1]
new_feat_test2 = new_feat_test2[:, :-1]

In [204]:
df_train["Credit_Product"] = df_train["Credit_Product"].fillna('TBD')
df_test["Credit_Product"] = df_test["Credit_Product"].fillna('TBD')

In [205]:


le = LabelEncoder()
df_train["Gender"] = le.fit_transform(df_train["Gender"])
df_test["Gender"] = le.transform(df_test["Gender"])

le = LabelEncoder()
df_train["Region_Code"] = le.fit_transform(df_train["Region_Code"])
df_test["Region_Code"] = le.transform(df_test["Region_Code"])

le = LabelEncoder()
df_train["Occupation"] = le.fit_transform(df_train["Occupation"])
df_test["Occupation"] = le.transform(df_test["Occupation"])

le = LabelEncoder()
df_train["Channel_Code"] = le.fit_transform(df_train["Channel_Code"])
df_test["Channel_Code"] = le.transform(df_test["Channel_Code"])

le = LabelEncoder()
df_train["Credit_Product"] = le.fit_transform(df_train["Credit_Product"])
df_test["Credit_Product"] = le.transform(df_test["Credit_Product"])

le = LabelEncoder()
df_train["Is_Active"] = le.fit_transform(df_train["Is_Active"])
df_test["Is_Active"] = le.transform(df_test["Is_Active"])


In [206]:
df_train.drop(['ID'], axis = 1, inplace= True)
Test_ID = df_test.ID.values
df_test.drop(['ID'], axis = 1, inplace= True)

In [207]:
#feature engineering

df_train['Potential'] = df_train['Age'] * df_train['Avg_Account_Balance']
df_test['Potential'] = df_test['Age'] * df_test['Avg_Account_Balance']

In [208]:
X_train, Y = df_train.drop(["Is_Lead"], axis=1).values, df_train["Is_Lead"].values
X_test = df_test.values

X_train = np.hstack((X_train,new_feat_train, new_feat_train2))
X_test = np.hstack((X_test,  new_feat_test, new_feat_test2))

In [209]:
oversample = SMOTE(sampling_strategy=1)
X_train, Y = oversample.fit_resample(X_train, Y)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


In [210]:
# oof

i=0
setused=X_train
targ=Y

scores=[]
splits=10

cnf_matrix=[]
oof_preds = np.zeros((len(X_test)))
kfold, scores = StratifiedKFold(n_splits=splits,random_state=True), list()
for train2, test2 in kfold.split(setused,targ):
    x_train, x_test = setused[train2], setused[test2]
    y_train, y_test = targ[train2], targ[test2]
    eval_set = [(x_test,y_test)]
    #x_train, y_train = oversample.fit_resample(x_train, y_train) 
    model = LGBMClassifier(random_state=27)

    model.fit(x_train, y_train)
    preds = model.predict_proba(x_test)[:, 1]
    score = roc_auc_score(y_test, preds)
    scores.append(score)
    print(score)
    oof_preds += model.predict_proba(X_test)[:, 1]
oof_preds = oof_preds/splits

print("Average: ", np.sum(scores)/len(scores))

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:296: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning


0.8720417521798752
0.87390191778427
0.8758480036132548
0.9846310130665913
0.9984330385606017
0.9985407299248599
0.998282144269698
0.998841335820146
0.9985060406333175
0.998488563629127
Average:  0.9597514539481742


In [ ]:
df_submit = pd.DataFrame({'ID':Test_ID , 'Is_Lead': oof_preds})
df_submit.to_csv("submit_jobathon_potential.csv", index=False)